In [ ]:
import torch
from torch import nn

# 데이터 작업을 위한 기본 요소 DataLoader, Dataset으로 구성
# DataLoader : Dataset을 순회 가능한 객체(iterable)로 감싼다.
# Dataset : 샘플과 레이블을 저장함.
#from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# torchvision.datasets 모듈은 CIFAR, COCO등과 같은 다양한 실제 비전 데이터에 대한 데이터를 포함한다.
from torchvision import datasets
from torchvision.transforms import ToTensor



In [ ]:
# FashionMNIST
# Train set download
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

# Test set download
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size = 64

# DataLoader
# 64 feature과 label을 batch로 반환
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
# Model design
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = DNN().to(device)
print(model)

DNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
learning_rate = 1e-3

#loss function, optimizer 정의
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        #backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss : {loss:>7f} [{current:>5d}/{size:>5d}]" )

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [ ]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------
loss : 2.171742 [    0/60000]
loss : 2.163291 [ 6400/60000]
loss : 2.101318 [12800/60000]
loss : 2.117511 [19200/60000]
loss : 2.063354 [25600/60000]
loss : 2.012370 [32000/60000]
loss : 2.038084 [38400/60000]
loss : 1.962670 [44800/60000]
loss : 1.964423 [51200/60000]
loss : 1.891713 [57600/60000]
Test error: 
 Accuracy: 61.6%, Avg loss: 0.029683

Epoch 2
-------------
loss : 1.918604 [    0/60000]
loss : 1.894250 [ 6400/60000]
loss : 1.769060 [12800/60000]
loss : 1.810399 [19200/60000]
loss : 1.696194 [25600/60000]
loss : 1.650066 [32000/60000]
loss : 1.670190 [38400/60000]
loss : 1.571868 [44800/60000]
loss : 1.591503 [51200/60000]
loss : 1.485442 [57600/60000]
Test error: 
 Accuracy: 63.4%, Avg loss: 0.023657

Epoch 3
-------------
loss : 1.569267 [    0/60000]
loss : 1.542246 [ 6400/60000]
loss : 1.384713 [12800/60000]
loss : 1.454998 [19200/60000]
loss : 1.332423 [25600/60000]
loss : 1.331210 [32000/60000]
loss : 1.341958 [38400/60000]
loss : 1.269802 [44800

In [ ]:
# model save
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

# model load
model = DNN()
model.load_state_dict(torch.load("model.pth"))

Saved PyTorch Model State to model.pth


<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
